In [2]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import re
import copy

def clean_excess_space(string_list):
    new_list = []
    for entry in string_list:
        new_list.append(re.sub(r"""\n\s{1,}""", " ", entry))
    
    return new_list

filenames = ['1', '2', '3']

for f in filenames:
    tables, headers = get_tables_from_xml_file(f)

def get_tables_from_xml_file(filepath):

    tree = ET.parse(filepath)
    root = ET.tostring(tree.getroot())
    content_list = []
    total_list = []
    sub_header = []
    all_tables = []
    all_headers = []
    counter = 0
    table_lists = tree.iter('{http://www.elsevier.com/xml/common/dtd}table')
    for table in table_lists:
        # get the head column
        header = []
        lst = table.iter('{http://www.elsevier.com/xml/common/cals/dtd}thead')
        for tags in lst:
            rows = tags.findall('{http://www.elsevier.com/xml/common/cals/dtd}row')
            entry_list = rows[0].findall('{http://www.elsevier.com/xml/common/dtd}entry')
            for every_entry in entry_list:
                total = ''
                for x in every_entry.itertext():
                    total = total + x
                total_list.append(total)
                if 'nameend' in every_entry.attrib.keys():
                    if every_entry.attrib['nameend'][:3] == 'col':
                        namest = int(every_entry.attrib['namest'][3:])
                        nameend = int(every_entry.attrib['nameend'][3:])
                        total_list = total_list + ['']*(nameend - namest)
            
            total_list = clean_excess_space(total_list)
            header.append(total_list)
            
            if len(rows) == 2:
                entry_list = rows[1].findall('{http://www.elsevier.com/xml/common/dtd}entry')
                for more_entry in entry_list: 
                    total2 = ''
                    for y in more_entry.itertext():
                        total2 =  total2 + y 
                    sub_header.append(total2)
                sub_header = clean_excess_space(sub_header)
                content_list.append(sub_header)
                header.append(sub_header)
            
        lst2 = table.iter('{http://www.elsevier.com/xml/common/cals/dtd}tbody')
        for data in lst2:
            rows = data.findall('{http://www.elsevier.com/xml/common/cals/dtd}row')
            for row in rows: 
                entry_list = row.findall('{http://www.elsevier.com/xml/common/dtd}entry')
                row_data = []
                for entry in entry_list:
                    content = entry.text 
                    row_data.append(content)
                content_list.append(clean_excess_space(row_data))
        
        rows = content_list
        df = pd.DataFrame(rows, columns = total_list)
        #print(df)
        #df.to_csv('Table' + str(counter) + '.csv',encoding='utf-8')
        counter = counter + 1
        content_list.clear()
        all_headers.append(copy.deepcopy(header))
        total_list.clear()
        sub_header.clear()
        all_tables.append(df)   
    
    return (all_tables, all_headers)

In [3]:
tables, headers = get_tables_from_xml_file('1.xml')

In [5]:
tables[0]

,Defect,ΔLBIC [%],First appearance,Defect identification
0,A,−41,Active Layer,Silica microsphere under active layer
1,B,−21,ITO,ITO defect or particle on ITO surface
2,C,7,Active Layer,Negative height defect in active layer
3,D,−73,Active Layer,Fibrous particle in/on/under active layer
4,E,6,ZnO,Radial ZnO ‘ripple’ from spin coating
5,F,84,ZnO,Large ZnO aggregate


In [7]:
headers[0][0]

[' Defect ', ' ΔLBIC [%] ', ' First appearance ', ' Defect identification ']

In [30]:
targeted_properties = ' ΔLBIC [%] '
targeted_materials = ' Defect '
#targeted_materials = ' DEfect'

In [23]:
if targeted_properties in headers[0][0]:
    info = tables[0][targeted_properties]

In [31]:
material = []
if targeted_materials in headers[0][0]:
    material = tables[0][targeted_materials]

In [32]:
material

0    A
1    B
2    C
3    D
4    E
5    F
Name:  Defect , dtype: object

In [14]:
info

0    −41
1    −21
2      7
3    −73
4      6
5     84
Name:  ΔLBIC [%] , dtype: object

In [19]:
material

0    A
1    B
2    C
3    D
4    E
5    F
Name:  Defect , dtype: object

In [20]:
output_table = pd.DataFrame({targeted_materials: material,
                             targeted_properties: info})

In [21]:
output_table

,Defect,ΔLBIC [%]
0,A,−41
1,B,−21
2,C,7
3,D,−73
4,E,6
5,F,84


In [130]:
tables, headers = get_tables_from_xml_file('2.xml')

In [131]:
headers

[[[' IFLs ',
   '',
   ' Device structures ',
   ' Device performance ',
   '',
   '',
   '',
   ' Ref. '],
  [' Type of materials ',
   ' Materials ',
   '',
   ' V OC (V) ',
   ' J SC (mA/cm 2 ) ',
   ' FF (%) ',
   ' PCE (%) ',
   '']],
 [[' IFLs ',
   '',
   ' Device structures ',
   ' Device performance ',
   '',
   '',
   '',
   ' Ref. '],
  [' Type of materials ',
   ' Materials ',
   '',
   ' V OC (V) ',
   ' J SC (mA/cm 2 ) ',
   ' FF (%) ',
   ' PCE (%) ',
   '']]]

In [80]:
tables[0]

,IFLs,,Device structures,Device performance,,,,Ref.
0,Type of materials,Materials,,V OC (V),J SC (mA/cm 2 ),FF (%),PCE (%),
1,,WPF-6-oxy-F,ITO/PEDOT:PSS/P3HT:PC,0.62,9.89,59.0,3.67,66
2,PF6NO25-py,ITO/PEDOT:PSS/PCDTBT:PC,0.91,11.60,66.2,6.90,68,None
3,PF6NO,ITO/PEDOT:PSS/PCDTBT:PC,0.93,7.27,62.8,6.07,69,None
4,PFNSO-BT,ITO/PEDOT:PSS/PTB7:PC,0.65,16.76,61.1,6.61,70,None
5,P3TMAHT,ITO/PEDOT:PSS/PCDTBT:PC,0.86,10.80,66.0,6.10,77,None
6,PF2/6-b-P3TMAHT,ITO/PEDOT:PSS/PCDTBT:PC,0.89,10.60,67.0,6.20,77,None
7,,bis-PC,ITO/PEDOT:PSS/PIDT-PhanQ:PC,0.88,11.19,60.0,5.87,81
8,bis-PC,ITO/PEDOT:PSS/PIDT-PhanQ:PC,0.88,11.50,61.0,6.22,81,None
9,ETL-1,ITO/PEDOT:PSS/PIDT-PhanQ:PC,0.87,11.28,64.0,6.28,82,None


In [81]:
tables[1].loc[0]

 IFLs                    Type of materials 
                                 Materials 
 Device structures                         
 Device performance               V OC (V) 
                           J SC (mA/cm 2 ) 
                                    FF (%) 
                                   PCE (%) 
 Ref.                                      
Name: 0, dtype: object